In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torchsummary import summary
import dsntnn

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        model = models.resnet18(pretrained=True)
        # change the first layer to recieve five channel image
        model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,bias=True)
        # change the last layer to output 32 coordinates
        # model.fc=nn.Linear(512,32)
        # remove final two layers(fc, avepool)
        model = nn.Sequential(*(list(model.children())[:-2]))
        for param in model.parameters():
            param.requires_grad = True
        self.resnet = model.cuda()
        
    def forward(self, x):
       
        pose_out = self.resnet(x)
        return pose_out

class CoordRegressionNetwork(nn.Module):
    def __init__(self, n_locations):
        super(CoordRegressionNetwork, self).__init__()
        self.resnet = Net()
        self.hm_conv = nn.Conv2d(512, n_locations, kernel_size=1, bias=False)

    def forward(self, images):
        # 1. Run the images through our Resnet
        resnet_out = self.resnet(images)
        # 2. Use a 1x1 conv to get one unnormalized heatmap per location
        unnormalized_heatmaps = self.hm_conv(resnet_out)
        # 3. Normalize the heatmaps
        heatmaps = dsntnn.flat_softmax(unnormalized_heatmaps)
        # 4. Calculate the coordinates
        coords = dsntnn.dsnt(heatmaps)

        return coords, heatmaps

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
model = CoordRegressionNetwork(n_locations=16).to(device)

summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [21]:
print(model(torch.Tensor(1,3,224,224).to(device))[0].shape)

torch.Size([1, 16, 2])


In [12]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print name, param.data

resnet.0.weight tensor([[[[ 7.6255e-02,  1.1283e-02,  4.5261e-02,  ..., -6.2951e-02,
           -4.6515e-02,  6.4946e-02],
          [-4.2114e-02,  2.4183e-02,  4.7061e-02,  ...,  7.6182e-02,
           -2.0447e-02,  6.4806e-02],
          [ 2.4900e-02, -6.8025e-02, -1.3969e-03,  ...,  8.1677e-02,
           -8.0341e-02, -1.2164e-02],
          ...,
          [ 6.8321e-02,  1.1614e-02, -8.0783e-02,  ..., -2.8551e-02,
           -1.3811e-02,  6.1159e-02],
          [-3.5042e-02,  4.4408e-03,  5.4944e-02,  ..., -4.4735e-02,
           -6.7446e-02,  5.8848e-02],
          [-3.5495e-02, -1.5472e-02, -7.0017e-02,  ..., -5.8220e-02,
            5.6371e-02,  5.0029e-02]],

         [[ 1.1197e-02,  2.7468e-03,  7.4212e-02,  ..., -6.8255e-02,
            6.1652e-02, -5.6035e-02],
          [-7.7118e-02,  6.2068e-02, -2.4604e-02,  ..., -1.3380e-02,
            6.0245e-02, -5.5002e-02],
          [ 8.0126e-04,  6.3657e-02,  4.6881e-03,  ..., -3.0583e-02,
           -4.1366e-02, -6.7596e-02],
    

resnet.6.0.conv2.weight tensor([[[[-0.0093, -0.0339, -0.0119],
          [-0.0246, -0.0798, -0.0487],
          [-0.0435, -0.0801, -0.0653]],

         [[-0.0289,  0.0002, -0.0286],
          [ 0.0099,  0.0103, -0.0177],
          [-0.0107,  0.0028, -0.0125]],

         [[-0.0147,  0.0226,  0.0044],
          [ 0.0155,  0.0109, -0.0040],
          [-0.0208, -0.0180, -0.0173]],

         ...,

         [[ 0.0003, -0.0041, -0.0008],
          [-0.0217, -0.0223, -0.0299],
          [ 0.0105,  0.0035, -0.0114]],

         [[ 0.0097,  0.0184,  0.0370],
          [ 0.0037,  0.0104,  0.0152],
          [ 0.0084,  0.0183,  0.0302]],

         [[ 0.0014,  0.0084,  0.0097],
          [ 0.0265,  0.0415,  0.0553],
          [ 0.0169,  0.0610,  0.0563]]],


        [[[ 0.0117,  0.0165,  0.0051],
          [ 0.0294,  0.0204,  0.0216],
          [ 0.0079,  0.0133,  0.0117]],

         [[-0.0153, -0.0213, -0.0090],
          [-0.0292, -0.0516, -0.0436],
          [-0.0045, -0.0372, -0.0420]],

       

In [16]:
print(model)

CoordRegressionNetwork(
  (resnet): Net(
    (resnet): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.